# pgRouting 2: Routing

In this exercise we are going to cover the following topics:
1. How to create a routable streetnetwork from preprocessed OpenStreetMap data.
2. How to create route stops from sample points.
3. How to solve the Travelling Salesman Problem (TSP) to visit the route stops in a cost efficient order.
4. How to compute the shortest path via all ordered route stops.

We will need the following software for this exercise:
1. PostgreSQL >9.1 with PostGIS >2.0 and pgRouting >2.4 extensions

We will need the following datasets in a PostgreSQL database for this exercise: 
1. Preprocessed OpenStreetMap street data (see "pgRouting 1: Data preparation").
2. Sample points (see "???").

Note: In this exercise we will execute SQL commands from within Python for convenience. You can also extract the SQL strings and run them directly through pgsql.

Author: Marc Wieland
Last modified: 10.05.2017

## Connect to database

In [1]:
import db

# connect to database
dbconn = db.Db("host='localhost' dbname='osmrouting_test' user='postgres' password='postgres'")

First we need to connect to the database that we have created in the previous exercise ("pgRouting 1: Data preparation").

## Create a routable streetnetwork from preprocessed OpenStreetMap data

In [3]:
# filter OpenStreetMap street data by tags on road type (this should give only car roads)
#dbconn.query("ALTER TABLE osm_streets ALTER COLUMN gid TYPE bigint;")
dbconn.query("""SELECT * INTO routing.osm_streets FROM public.santiago_roads WHERE
                    fclass = 'motorway' OR 
                    fclass = 'motorway_link' OR 
                    fclass = 'primary' OR 
                    fclass = 'primary_link' OR 
                    fclass = 'primary;primary' OR 
                    fclass = 'residential' OR 
                    fclass = 'road' OR 
                    fclass = 'secondary' OR 
                    fclass = 'secondary_link' OR 
                    fclass = 'tertiary' OR 
                    fclass = 'tertiary_link' OR 
                    fclass = 'unclassified' OR 
                    fclass ='living_street' OR
                    fclass= 'trunk' OR 
                    fclass = 'trunk_link';""")


# add id, primary key, index and cluster analyze for faster spatial queries
dbconn.query("ALTER TABLE routing.osm_streets ADD COLUMN id integer;")
dbconn.query("UPDATE routing.osm_streets SET id=gid;")
dbconn.query("ALTER TABLE routing.osm_streets ADD PRIMARY KEY (gid);")
dbconn.query("CREATE INDEX osm_streets_gix ON routing.osm_streets USING GIST (geom);")
dbconn.query("CLUSTER routing.osm_streets USING osm_streets_gix;")
dbconn.query("ANALYZE routing.osm_streets;")

The code above selects all records in our streets table that are marked with tags that refer to roads which can be accessed with a car. This effectively filters out footpaths, cycle lanes or any other street types that can not be accessed with a car. A comprehensive list of OpenStreetMap tags and background on the how the OpenStreetMap tagging system works can be found here: https://taginfo.openstreetmap.org

In [4]:
# create routable street network (note: this can take several minutes for large datasets)
dbconn.queryRes("SELECT * FROM pgr_createnetwork('routing.osm_streets');")

# rename table
dbconn.query("ALTER TABLE routing.osm_streets_vertices_pgr RENAME COLUMN the_geom TO geom;")

The code above creates a routable street network. This involves a number of processing steps convenientaly wrapped into the custom *pgr_createnetwork()* function. 

First, we have to make sure that the data provides a correct network topology, which consists of information about **source and target ID of each street edge**. Thus, the function adds *source* and *target* columns for each street edge. 

Second, we need a **cost attribute** that defines the cost of travelling any given street edge. The function adds a *cost* column and calculates as default cost attribute the length of each street edge (in meters).

Third, we need to build a **network topology** for our street data. This means is that for any given edge in the street data the ends of that edge will be connected to a unique node and to other edges that are also connected to that same unique node. Once all the edges are connected to nodes we have a graph that can be used for routing with pgrouting. The pgRouting network topology is represented by an edge table (the *osm_streets* table with source, target and cost attributes) and a vertices table associated with it (created on the fly).

## Create route stops from sample points.

In [6]:
# rename geometry column if generated by OGR driver
#dbconn.query("ALTER TABLE public.samples_pps50 RENAME COLUMN wkb_geometry TO geom;")

# create route stops from sample points
dbconn.queryRes("SELECT * FROM pgr_createroutestops('routing.osm_streets_vertices_pgr', 'public.samples_pps50', 50);")

# rename table, add primary key and index
dbconn.query("ALTER TABLE routing.route_stops RENAME TO samples_pps50;")
dbconn.query("ALTER TABLE routing.samples_pps50 ADD PRIMARY KEY (id);")
dbconn.query("CREATE INDEX samples_pps50_gix ON routing.samples_pps50 USING GIST (geom);")

The code above creates route stops from sample points. The custom *pgr_createroutestops()* function selects the nearest node to any sample point on the street network and deletes any duplicate nodes from the table, since it can happen that several sample points have the same nearest node.

## Solve the Travelling Salesman Problem (TSP) to visit the route stops in a cost efficient order.

In [7]:
# order route stops using TSP
dbconn.query("""SELECT a.seq, a.agg_cost, b.* INTO routing.samples_pps50_tsp FROM pgr_tsp(
                    $$
                    SELECT * FROM pgr_dijkstraCostMatrix(
                        'SELECT id, source, target, cost, reverse_cost FROM routing.osm_streets', 
                        (SELECT array_agg(node) FROM routing.samples_pps50), 
                        directed := false
                    )
                    $$,
                    start_id := 86246,
                    randomize := false
                ) a LEFT JOIN routing.samples_pps50 b ON (a.node = b.node);""")

The **Travelling Salesperson Problem (TSP)** asks the question

*"Given a list of cities and the distances between each pair of cities, what is the shortest possible route that visits each city exactly once and returns to the origin city?"*

The code above orders the route stops so that a traveller visits all the nodes exactly once in a cost optimized way. The pgRouting implementation *pgr_tsp()* uses simulated annealing to return the approximate solution when the input is given in the form of matrix cell contents. This means that it needs as input a **cost matrix**, which in our case is calculated using the *pgr_dijkstraCostMatrix()* function with the cost attribute being the length of each street edge (as calculated previously). Optionally a start point (*start_id*) can be defined. This is the ID of the node from which the route shall start.

# 4. Compute the shortest path via all ordered route stops.

In [ ]:
# compute shortest path across all ordered route stops
dbconn.queryRes("SELECT * FROM pgr_dijkstramulti('routing.samples_pps50_tsp', 'routing.osm_streets');")

# rename table, add primary key and index
dbconn.query("ALTER TABLE routing.route_dijkstramulti RENAME TO samples_pps50_tsp_route;")
dbconn.query("ALTER TABLE routing.samples_pps50_tsp_route ADD COLUMN gid serial;")
dbconn.query("ALTER TABLE routing.samples_pps50_tsp_route ADD PRIMARY KEY (gid);")
dbconn.query("CREATE INDEX samples_pps50_tsp_route_gix ON routing.samples_pps50_tsp_route USING GIST (geom);")

The code above returns the shortest path via all ordered route stops using the **Dijkstra algorithm**. Dijkstra’s algorithm, conceived by Dutch computer scientist Edsger Dijkstra in 1956, is a graph search algorithm that solves the shortest path problem for a graph with non-negative edge path costs, producing a shortest path from a starting vertex (start_vid) to an ending vertex (end_vid). In our case need to return the shortest path multiple times between each pair of succeeding vertices along the ordered sequence of ordered route stops.

In [12]:
# close database connection
del dbconn

Finally we close the database connection.